# 编译 XS-GEM5  Complie XS-GEM5

这里我们将展示XS-GEM5的编译流程

Here we will demonstrate the compilation process of XS-GEM5

首先使用 scons 编译 GEM5。

`--linker=mold` 是可选的，可以加速编译过程，需要事先安装 mold 链接器，我们提供的 code-server 镜像中已经包含。

以上命令最后会生成 gem5.opt文件。

First, we can use scons to compile GEM5.

The option --linker=mold is optional and can speed up the compilation process
You need to install the mold linker beforehand, but it is already included in our code-server image.

The above command will generate the gem5.opt file.

In [ ]:
%%bash
cd .. && source env.sh
cd ${GEM5_HOME}

#scons build/RISCV/gem5.opt --linker=mold -j8